In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

In [2]:
DATADIR = 'Datasets/Train'
CATEGORIES = ['Fire', 'NoFire']

In [3]:
IMG_SIZE = 128
def create_training_data():
    training_data = []
    for category in CATEGORIES:  
        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
    return training_data

In [4]:
training_data = create_training_data()

100%|██████████| 1301/1301 [00:29<00:00, 43.67it/s]


In [5]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0

X.shape[1:]

(128, 128, 3)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.3)

In [7]:
# set up image augmentation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    )

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    )

val_datagen.fit(X_val)

Using TensorFlow backend.


In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W1213 13:22:17.959485 139763806406464 deprecation.py:506] From /home/hssahrul/miniconda3/envs/ML/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
history = model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=32), 
    validation_data=val_datagen.flow(X_val, y_val, batch_size=32), 
    epochs=100, 
    verbose=1
)

Epoch 1/100
53/53 [==============================] - 48s 909ms/step - loss: 0.5801 - acc: 0.6916 - val_loss: 0.5056 - val_acc: 0.7785
Epoch 2/100
53/53 [==============================] - 37s 691ms/step - loss: 0.4965 - acc: 0.7689 - val_loss: 0.4510 - val_acc: 0.8047
Epoch 3/100
53/53 [==============================] - 36s 670ms/step - loss: 0.4336 - acc: 0.8001 - val_loss: 0.4962 - val_acc: 0.7923
Epoch 4/100
53/53 [==============================] - 35s 658ms/step - loss: 0.4309 - acc: 0.8060 - val_loss: 0.4635 - val_acc: 0.7992
Epoch 5/100
53/53 [==============================] - 36s 672ms/step - loss: 0.3993 - acc: 0.8219 - val_loss: 0.4686 - val_acc: 0.8074
Epoch 6/100
53/53 [==============================] - 36s 677ms/step - loss: 0.3700 - acc: 0.8325 - val_loss: 0.4251 - val_acc: 0.8129
Epoch 7/100
53/53 [==============================] - 36s 675ms/step - loss: 0.3533 - acc: 0.8396 - val_loss: 0.4643 - val_acc: 0.8033
Epoch 8/100
53/53 [==============================] - 40s 750ms

KeyboardInterrupt: 

In [ ]:
model_name = 'Fire-128x128-CNN-v2'
model.save(f'Model/{model_name}.h5')

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title(f'model accuracy of {model_name}')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.savefig(f'{model_name}-accuracy')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title(f'model loss of {model_name}')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.savefig(f'{model_name}-loss')
plt.show()